<a href="https://colab.research.google.com/github/hwangdonggeun/project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pickle
from datetime import datetime

def get_budget_folder_path():
    budget_folder = os.path.join(os.path.expanduser("~"), 'budget')
    if not os.path.exists(budget_folder):
        os.makedirs(budget_folder)
    return budget_folder

def update_existing_transactions():
    budget_folder_path = get_budget_folder_path()
    transactions_file = os.path.join(budget_folder_path, 'transactions.pkl')

    if os.path.exists(transactions_file):
        with open(transactions_file, 'rb') as file:
            transactions = pickle.load(file)

        updated = False
        for transaction in transactions:
            if 'category' not in transaction:
                transaction['category'] = 'Uncategorized'
                updated = True

        if updated:
            with open(transactions_file, 'wb') as file:
                pickle.dump(transactions, file)
            print("Existing transactions updated with default category.")

def add_transaction(transaction_type, amount, date_str, category='Uncategorized'):
    budget_folder_path = get_budget_folder_path()
    transactions_file = os.path.join(budget_folder_path, 'transactions.pkl')

    if not os.path.exists(transactions_file):
        transactions = []
    else:
        with open(transactions_file, 'rb') as file:
            transactions = pickle.load(file)

    try:
        date = datetime.strptime(date_str, '%Y-%m-%d').strftime('%Y-%m-%d %H:%M:%S')
    except ValueError:
        print("Invalid date format. Please use YYYY-MM-DD.")
        return

    transaction = {
        'type': transaction_type,
        'amount': amount,
        'category': category,
        'date': date
    }
    transactions.append(transaction)

    with open(transactions_file, 'wb') as file:
        pickle.dump(transactions, file)

    print(f"{transaction_type.capitalize()} of {amount} added on {date_str} to category {category}.")

def generate_category_report(target_month):
    budget_folder_path = get_budget_folder_path()
    transactions_file = os.path.join(budget_folder_path, 'transactions.pkl')

    if not os.path.exists(transactions_file):
        print("No transactions found.")
        return

    with open(transactions_file, 'rb') as file:
        transactions = pickle.load(file)

    category_report = {}
    total_income = 0
    total_expense = 0

    for transaction in transactions:
        month = transaction['date'][:7]
        if month == target_month:
            category = transaction['category']
            if category not in category_report:
                category_report[category] = {'income': 0, 'expense': 0}

            if transaction['type'] == 'income':
                category_report[category]['income'] += transaction['amount']
                total_income += transaction['amount']
            elif transaction['type'] == 'expense':
                category_report[category]['expense'] += transaction['amount']
                total_expense += transaction['amount']

    if total_income == 0 and total_expense == 0:
        print(f"No transactions found for {target_month}.")
        return

    for category, data in category_report.items():
        print(f"Category: {category} - Income: {data['income']}, Expense: {data['expense']}, Balance: {data['income'] - data['expense']}")

    print(f"Total Income for {target_month}: {total_income}")
    print(f"Total Expense for {target_month}: {total_expense}")
    print(f"Overall Balance for {target_month}: {total_income - total_expense}")

def delete_transactions(target_month=None):
    budget_folder_path = get_budget_folder_path()
    transactions_file = os.path.join(budget_folder_path, 'transactions.pkl')

    if not os.path.exists(transactions_file):
        print("No transactions found.")
        return

    with open(transactions_file, 'rb') as file:
        transactions = pickle.load(file)

    if target_month:
        transactions = [transaction for transaction in transactions if transaction['date'][:7] != target_month]
        print(f"All transactions for {target_month} have been deleted.")
    else:
        transactions = []
        print("All transactions have been deleted.")

    with open(transactions_file, 'wb') as file:
        pickle.dump(transactions, file)

def calculate_monthly_averages(target_month):
    budget_folder_path = get_budget_folder_path()
    transactions_file = os.path.join(budget_folder_path, 'transactions.pkl')

    if not os.path.exists(transactions_file):
        print("No transactions found.")
        return

    with open(transactions_file, 'rb') as file:
        transactions = pickle.load(file)

    monthly_data = {'income': 0, 'expense': 0}
    for transaction in transactions:
        month = transaction['date'][:7]
        if month == target_month:
            if transaction['type'] == 'income':
                monthly_data['income'] += transaction['amount']
            elif transaction['type'] == 'expense':
                monthly_data['expense'] += transaction['amount']

    print(f"Total Income for {target_month}: {monthly_data['income']}")
    print(f"Total Expense for {target_month}: {monthly_data['expense']}")

def calculate_savings_rate(target_month):
    budget_folder_path = get_budget_folder_path()
    transactions_file = os.path.join(budget_folder_path, 'transactions.pkl')

    if not os.path.exists(transactions_file):
        print("No transactions found.")
        return

    with open(transactions_file, 'rb') as file:
        transactions = pickle.load(file)

    total_income = 0
    total_expense = 0
    for transaction in transactions:
        month = transaction['date'][:7]
        if month == target_month:
            if transaction['type'] == 'income':
                total_income += transaction['amount']
            elif transaction['type'] == 'expense':
                total_expense += transaction['amount']

    if total_income == 0:
        print("No income to calculate savings rate.")
        return

    savings_rate = ((total_income - total_expense) / total_income) * 100
    print(f"Savings Rate for {target_month}: {savings_rate:.2f}%")

def calculate_category_spending_ratio(target_month):
    budget_folder_path = get_budget_folder_path()
    transactions_file = os.path.join(budget_folder_path, 'transactions.pkl')

    if not os.path.exists(transactions_file):
        print("No transactions found.")
        return

    with open(transactions_file, 'rb') as file:
        transactions = pickle.load(file)

    total_expense = 0
    category_expense = {}
    for transaction in transactions:
        month = transaction['date'][:7]
        if month == target_month and transaction['type'] == 'expense':
            total_expense += transaction['amount']
            category = transaction['category']
            if category not in category_expense:
                category_expense[category] = 0
            category_expense[category] += transaction['amount']

    if total_expense == 0:
        print("No expenses to calculate category spending ratio.")
        return

    for category, amount in category_expense.items():
        ratio = (amount / total_expense) * 100
        print(f"Category: {category} - Spending Ratio for {target_month}: {ratio:.2f}%")

update_existing_transactions()

while True:
    print("1. Add Income")
    print("2. Add Expense")
    print("3. Delete Transactions")
    print("4. Generate Category Report")
    print("5. Calculate Monthly Averages")
    print("6. Calculate Savings Rate")
    print("7. Calculate Category Spending Ratio")
    print("8. Exit")
    choice = input("Choose an option: ")

    if choice == '1':
        amount = float(input("Enter amount: "))
        date_str = input("Enter date (YYYY-MM-DD): ")
        category = input("Enter category: ")
        add_transaction('income', amount, date_str, category)
    elif choice == '2':
        amount = float(input("Enter amount: "))
        date_str = input("Enter date (YYYY-MM-DD): ")
        category = input("Enter category: ")
        add_transaction('expense', amount, date_str, category)
    elif choice == '3':
        target_month = input("Enter target month (YYYY-MM) or press Enter to delete all transactions: ")
        delete_transactions(target_month if target_month else None)
    elif choice == '4':
        target_month = input("Enter target month (YYYY-MM): ")
        generate_category_report(target_month)
    elif choice == '5':
        target_month = input("Enter target month (YYYY-MM): ")
        calculate_monthly_averages(target_month)
    elif choice == '6':
        target_month = input("Enter target month (YYYY-MM): ")
        calculate_savings_rate(target_month)
    elif choice == '7':
        target_month = input("Enter target month (YYYY-MM): ")
        calculate_category_spending_ratio(target_month)
    elif choice == '8':
        break
    else:
        print("Invalid choice. Please try again.")


1. Add Income
2. Add Expense
3. Delete Transactions
4. Generate Category Report
5. Calculate Monthly Averages
6. Calculate Savings Rate
7. Calculate Category Spending Ratio
8. Exit
